In [ ]:
#Dataset pre-prossesing
#Step 1: Read csv file and create a dataframe
#Step 2: Save dataframes into pkl files
#Step 3: Pre-processing 
#Step 4: Save pre-processed data

In [4]:
import pandas as pd
import pickle as pkl
import re
import spacy
import unicodedata
import greek_stemmer
import emoji
import spacy
import el_core_news_sm
import string 
import numpy as np
from greek_stemmer import GreekStemmer
from collections import Counter
from sklearn.metrics import cohen_kappa_score
string.punctuation
nlp = el_core_news_sm.load()

In [5]:
annotation = "AllData.csv"

c_l = []
everything = []

chunksize = 100
    
for chunk in pd.read_csv(annotation,engine='python',encoding = 'utf8',sep=";",header=1, usecols = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], chunksize=chunksize):
    
    r = chunk
    r.columns = ["Channel","VideoTitle","VideoLink","VideoPublishedYear","VideoViews","CommentID","ParrentCommentID","CommentAuthor","CommentText","CommentYear","Label"]
#     print(r.CommentText, r.Label)
    for i in range(len(chunk)):
        c_l.append({'CommentText': r.iloc[i]['CommentText'], 'Label': r.iloc[i]['Label']})
        everything.append({'Channel': r.iloc[i]['Channel'],'VideoTitle': r.iloc[i]['VideoTitle'],'VideoLink': r.iloc[i]['VideoLink'],'VideoPublishedYear': r.iloc[i]['VideoPublishedYear'],'VideoViews': r.iloc[i]['VideoViews'],'CommentID': r.iloc[i]['CommentID'],'ParrentCommentID': r.iloc[i]['ParrentCommentID'],'CommentAuthor': r.iloc[i]['CommentAuthor'],'CommentText': r.iloc[i]['CommentText'],'CommentYear': r.iloc[i]['CommentYear'],'Label': r.iloc[i]['Label']})
        
Commnet_Label = pd.DataFrame(c_l)
All = pd.DataFrame(everything)

In [6]:
Commnet_Label = pd.DataFrame(c_l)
All = pd.DataFrame(everything)

with open('Commnet_Label.pkl', 'wb') as handle:
    pkl.dump(Commnet_Label, handle)
    
with open('All.pkl', 'wb') as handle:
    pkl.dump(All, handle)

# Read pkl file

In [7]:
with open('Commnet_Label.pkl', 'rb') as handle:
    Comment_Label = pkl.load(handle)
    
with open('All.pkl', 'rb') as handle:
    All = pkl.load(handle)

In [3]:
Comment_Label.Label.value_counts()

Explicit        7638
Implicit        5459
No hate         4113
Can’t define    1152
Noise            874
Name: Label, dtype: int64

# PREPROCESSING

# Dataframe with only Explict and No Hate comments

In [ ]:
Expicit_NoHate = Comment_Label.loc[(Comment_Label['Label'] == "No hate") | (Comment_Label['Label'] == "Explicit")]

## Remove links, \n char \ufeff, @

In [ ]:
Expicit_NoHate.reset_index(drop=True, inplace=True)

NoLinks = Expicit_NoHate['CommentText'].str.replace('http\S+|www.\S+', '', case=False)
Expicit_NoHate['NoLink'] = NoLinks

#Remove \n char \ufeff, @

Expicit_NoHate['NoLink'] = Expicit_NoHate['NoLink'].str.replace('\n', '', case=False)
Expicit_NoHate['NoLink'] = Expicit_NoHate['NoLink'].str.replace('\ufeff', '', case=False)
Expicit_NoHate['NoLink'] = Expicit_NoHate['NoLink'].str.strip()

Expicit_NoHate['NoAuthor']= Expicit_NoHate['NoLink'].str.replace(r'^@', '', case=False)


# Remove usernames

In [ ]:
def delete_usernames(df_text,df_users):
    new_df=[]

    unique_authors = df_users.unique()
    unique_authors = list(unique_authors)
    unique_authors.sort(key = len, reverse=True)

    for i, x in enumerate(df_text):


        for author in unique_authors:
            x=x.replace(author, '')
        new_df.append(x)

    obj_df = pd.DataFrame(new_df)

    return obj_df

In [ ]:
Expicit_NoHate['NoAuthor'] = delete_usernames(Expicit_NoHate['NoAuthor'],All["CommentAuthor"])

# Create new column with emojie name and count dict

In [ ]:
def emojie(df):
    new_df=[]
    
    for row in df:
        emojie=[]
        
        
        for a in emoji.emoji_lis(row):
                    
                emojie_text = emoji.demojize(a["emoji"])
                emojie.append(emojie_text)
                
                
        if not emojie:  
            new_df.append(None)
        else:
            emojie=dict(Counter(emojie))
            new_df.append(emojie)
            
            
    obj_df = pd.DataFrame(new_df)
    return obj_df

In [ ]:
Expicit_NoHate["Emojie"] = emojie(Expicit_NoHate["NoAuthor"])

# Upercase

In [ ]:
Expicit_NoHate["Upper"] = Expicit_NoHate["NoAuthor"].str.upper()

# Remove whitespace


In [ ]:
Expicit_NoHate["Preprocessed"]=Expicit_NoHate["Upper"].str.strip()

# Remove accents

In [ ]:
#link github : https://gist.github.com/rcalsaverini/30bb8212809d29592222

def strip_accents(unicode_string):

    ndf_string = unicodedata.normalize('NFD', unicode_string)
    is_not_accent = lambda char: unicodedata.category(char) != 'Mn'
    return ''.join(
        char for char in ndf_string if is_not_accent(char) 
    )


In [ ]:
Expicit_NoHate["Upper"]=Expicit_NoHate["Upper"].apply(strip_accents)


# No punctuation

In [ ]:
def NoPunctuation(s):
    
    exclude = set(string.punctuation)
    s = ''.join(ch for ch in s if ch not in exclude)
    
    return s

# Greeklish to Greek

In [ ]:
def get_conversion_pool():
    poolGR = u"ΑΒΓΔΕΖΗΘΙΚΛΜΝΞΟΠΡΣΤΥΦΧΨΩΥΚΒ"
    poolGL =  "ABGDEZH8IKLMN3OPRSTUFX4WYCV"
    return dict(zip(poolGL, poolGR))

def convert(st):
    pool = get_conversion_pool()
    r=''
    
    s = u'{}'.format(st)
    s = s.replace('THN',u'ΤΗΝ')
    s = s.replace('THS',u'ΤΗΣ')
    s = s.replace('THA',u'ΘΑ')
    s = s.replace('J',u'TZ')
    s = s.replace('HA',u'ΧΑ')
    s = s.replace('KS',u'Ξ')
    s = s.replace('TH',u'Θ')
    s = s.replace('PS',u'Ψ')
    

    for i in range (len(s)):
        line = s[i]
        output_line = []
        
        for character in line:
        
            if character in pool:
                output_line.append(pool[character])
            else:
                output_line.append(character)

        r = r + "".join(output_line)
    return(r)

In [ ]:
Expicit_NoHate["Greek"]=Expicit_NoHate["Upper"].apply(convert)

# Remove stopwords

In [ ]:
def r_stopwords(text):
    doc = nlp(text)

    r=''
    for token in doc:
        output_line = []
        if token.is_stop:

            output_line.append("") 


        else:
    #         print(token)
            token=str(token)
            output_line.append(token+' ')

        r = r + " ".join(output_line)
    return(re.sub(' +', ' ',r))

In [ ]:
Expicit_NoHate["NoStopwords"]=Expicit_NoHate["Greek"].apply(r_stopwords)
lower_no_stop = Expicit_NoHate["NoStopwords"].str.lower()
Expicit_NoHate["NoStopwords"] = lower_no_stop.apply(r_stopwords)
Expicit_NoHate['NoStopwords'] = Expicit_NoHate['NoStopwords'].str.strip()

In [ ]:
Expicit_NoHate.NoStopwords = Expicit_NoHate.NoStopwords.str.upper()

# Tokenize

In [ ]:
def tokenize(text):
    doc = nlp(text)
    return[token.text for token in doc]

In [ ]:
Expicit_NoHate["Tokens"] = Expicit_NoHate["NoStopwords"].apply(tokenize)

# POS

In [ ]:
def POS(text):
    
    
    l=[]
    text = text.lower()
    doc = nlp(text)

    for word in doc:
        if emoji.emoji_lis(str(word)):
            pos = emoji.demojize(str(word))

        else:
            pos = str(word.pos_)
        l.append(pos)
    return l

In [ ]:
Expicit_NoHate['POS'] = Expicit_NoHate.NoStopwords.apply(POS)

# Stemming

In [ ]:
def GR_stemmer(text):
    
    doc = nlp(text)
    stemmer = GreekStemmer()
    r=''
    
    for token in doc:
        output_line = []
        token = str(token)
        stemm = stemmer.stem(token)
        

        output_line.append(stemm+' ')

        r = r + " ".join(output_line)
    return(re.sub(' +', ' ',r))

In [ ]:
Expicit_NoHate['Stemming'] =  Expicit_NoHate.NoStopwords.apply(GR_stemmer)

In [ ]:
def laugh(text):

    r=''
    
    regex = re.compile('[ΑΧ]+')
    punct = re.findall(r"[\w']+|[.,!?;]", text)

    for token in punct:
        output_line = []
                    
        if bool(re.fullmatch(regex, token)):

            output_line.append("XAXA"+" ") 


        else:
            token=str(token)
            output_line.append(token+' ')

        r = r + " ".join(output_line)
    return(re.sub(' +', ' ',r))

Expicit_NoHate["NoLaugh"]=Expicit_NoHate["NoStopwords"].apply(laugh)

In [ ]:
Expicit_NoHate.NoLaugh = Expicit_NoHate.NoLaugh.str.replace('\d+', '')

In [ ]:
Expicit_NoHate["NoLaughTokens"] = Expicit_NoHate.NoLaugh.apply(tokenize)

tokens_no_laugh = []

for token in Expicit_NoHate["NoLaughTokens"]:
    tokens_no_laugh.append(token)

In [ ]:
Expicit_NoHate["NoPunctuation"]=Expicit_NoHate["NoLaugh"].apply(NoPunctuation)

In [ ]:
Expicit_NoHate["NoLaughTokens_l"] = Expicit_NoHate.NoLaugh.str.lower()
Expicit_NoHate["NoLaughTokens_l"] = Expicit_NoHate.NoLaughTokens_l.apply(tokenize)

In [ ]:
Expicit_NoHate["StemTokens"] = Expicit_NoHate.Stemming.apply(tokenize)

In [ ]:
with open('Preprocessed.pkl', 'rb') as handle:
    Preprocessed = pkl.load(handle)

In [ ]:
#https://www.kaggle.com/baghern/a-deep-dive-into-sklearn-pipelines

def processing(df):
    
    new_df = pd.DataFrame()

    
    new_df['Length'] = df.apply(lambda x: len(x))

    new_df['Words'] = df.str.split().str.len()

    new_df['Avg_word_length'] = df.apply(lambda x: round(np.mean([len(t) for t in x.split(' ')]) if len([len(t) for t in x.split(' ')]) > 0 else 0, 2))

    new_df['Punctuation'] =  df.apply(lambda x: sum(1 for c in x if c in string.punctuation))
    
    new_df['Upper_count'] = df.apply(lambda x: sum(1 for c in x if c.isupper()))
    
    new_df['Upper_ratio'] = round((new_df['Upper_count'] * 100 )/ new_df['Length'], 1 )
    
    
    return new_df

numerical_features = processing(Preprocessed.NoAuthor)


In [ ]:
with open('Preprocessed.pkl', 'wb') as handle:
    pkl.dump(Expicit_NoHate, handle)

In [ ]:
with open('Numerical_Features.pkl', 'wb') as handle:
    pkl.dump(numerical_features, handle)

In [2]:
with open('Preprocessed.pkl', 'rb') as handle:
    Preprocessed = pkl.load(handle)